In [173]:
import sys
import os
import time
import datetime

import numpy as np
import scipy.ndimage
import pandas as pd
import matplotlib.pyplot as plt

from osgeo import gdal
from osgeo import osr
from osgeo import ogr
from osgeo.gdalconst import *
from netCDF4 import Dataset
import xarray as xr
#driver = gdal.GetDriverByName('GTiff')
#driver.Register()
driver = gdal.GetDriverByName('netCDF')
driver.Register()
driver_shp = ogr.GetDriverByName("ESRI Shapefile")

In [77]:
dir_path = os.path.realpath('./')
cwd = os.getcwd()

#################################### change watershed names/folders to area of interest! #########################
#global watershed_name, watershed_foldername, watershed_nick, working_dir, coords_system
#watershed_name = "Sleeper's"
watershed_foldername = "AORC_APCP_4KM_NERFC_201704"
#watershed_nick = "hub7"
working_dir = (os.path.abspath(os.path.join(dir_path,'..')) + "/data/" + 
               watershed_foldername + os.path.sep)
print(working_dir)

sleepers_box = [734500, 4924300, 7352300, 4924950] #useless
sleepers_coordinates = [[-72.054, 44.44], [-72.045,44.44]] 
originX, originY = -79.4853540000408429, 48.2488699999999966 #nc4 files upper left coordinates
resol = 0.03333199999997304985 #resolution in degrees

precip_folders = os.listdir(working_dir)
for file in precip_folders:
    if 'a' not in file:
        precip_folders.remove(file)
for folder in precip_folders:
    precip_files = os.listdir(working_dir+folder)
    for precip_file in precip_files:
        if  precip_file[-3:] != 'nc4':
            continue
        precip_nc = working_dir+folder+os.path.sep+precip_file
        print(precip_nc)
        print("I am analyzing {}".format(precip_nc))
        #os.system("gdalinfo " + precip_file)
        #precip_tif = precip_nc[:-3]+'tif'
        precip_tif = precip_nc.replace('.nc4','fede_geotiff')
        os.system(f'gdal_translate -b 1 {precip_nc} -of Gtiff {precip_tif}')
        #-s_srs EPSG:4326 -t_srs EPSG:26918 -te 734500 4924300 7352300 4924950 
        #os.system("gdal_translate -of GTiff -ot Float32 "+
        #          "-projwin_srs "+ precip_file + ' ' + precip_file[:-3]+'tif')
        
        precip_ds = gdal.Open(precip_tif, GA_ReadOnly)
        band = precip_ds.GetRasterBand(1)
        cols = precip_ds.RasterXSize
        rows = precip_ds.RasterYSize
        print(rows, cols)
        precipValues = band.ReadAsArray(0, 0, cols, rows)
        print(precipValues)
        geotransform = precip_ds.GetGeoTransform()
        originX = geotransform[0]
        originY = geotransform[3]
        raster_origin = [originX, originY]
        resol = geotransform[1]
        print(geotransform)
        sleepersCells = []
        for point in sleepers_coordinates:
            sleepersCells.append([-int((point[1] - originY) / resol), int((point[0] - originX) / resol)])
        #sleepersCells = np.array(sleepersCells)
        print(sleepersCells)
        sleepers_precipValues = []
        for point in sleepersCells:
            cellX, cellY = point[0], point[1]
            sleepers_precipValues.append(precipValues[cellX][cellY])
        print(sleepers_precipValues)
        
        break

    break
print("I am done.")
    
    
#dem = working_dir + watershed_nick + "_dem.tif"
#tanbeta = working_dir + watershed_nick + "_tanbeta.tif"
#ldd = working_dir + watershed_nick + "_ldd.tif"


C:\Users\feder\Documents\CUAHSI\repos\Aquaholics_Anonymous/data/AORC_APCP_4KM_NERFC_201704\
C:\Users\feder\Documents\CUAHSI\repos\Aquaholics_Anonymous/data/AORC_APCP_4KM_NERFC_201704\all\AORC_APCP_NERFC_2017040100.nc4
I am analyzing C:\Users\feder\Documents\CUAHSI\repos\Aquaholics_Anonymous/data/AORC_APCP_4KM_NERFC_201704\all\AORC_APCP_NERFC_2017040100.nc4
240 390
[[     9      9      9 ... -32767 -32767 -32767]
 [    10     10     10 ... -32767 -32767 -32767]
 [    11     10     11 ... -32767 -32767 -32767]
 ...
 [     0      0      0 ... -32767 -32767 -32767]
 [     0      0      0 ... -32767 -32767 -32767]
 [     0      0      0 ... -32767 -32767 -32767]]
(0.0, 1.0, 0.0, 0.0, 0.0, 1.0)
[[-44, -72], [-44, -72]]
[0, 0]
I am done.


In [42]:
for folder in precip_folders:
    precip_files = os.listdir(working_dir+folder)
    for precip_file in precip_files:
        precip_file = working_dir+folder+os.path.sep+precip_file
        print(precip_file)
        precipValues = Dataset(precip_file, "r", format='NETCDF4')
        print(precipValues)

C:\Users\feder\Documents\CUAHSI\repos\Aquaholics_Anonymous/data/VT_2017_Mar_Dec\AORC_APCP_4KM_NERFC_201704\AORC_APCP_NERFC_2017040100.nc4


OSError: [Errno -101] NetCDF: HDF error: b'C:\\Users\\feder\\Documents\\CUAHSI\\repos\\Aquaholics_Anonymous/data/VT_2017_Mar_Dec\\AORC_APCP_4KM_NERFC_201704\\AORC_APCP_NERFC_2017040100.nc4'

In [ ]:
dir_path = os.path.realpath('./')
cwd = os.getcwd()

#################################### change watershed names/folders to area of interest! #########################
#global watershed_name, watershed_foldername, watershed_nick, working_dir, coords_system
#watershed_name = "Sleeper's"
watershed_foldername = "VT_2017_Mar_Dec"
#watershed_nick = "hub7"
working_dir = (os.path.abspath(os.path.join(dir_path,'..')) + "/data/" + 
               watershed_foldername + os.path.sep)
print(working_dir)

hub3_precip_dict = {}
index = 0
precip_folders = os.listdir(working_dir)
for file in precip_folders:
    if 'AORC' not in file:
        precip_folders.remove(file)
for folder in precip_folders:
    precip_files = os.listdir(working_dir+folder)
    for precip_file in precip_files:
        if  precip_file[-3:] != 'nc4':
            continue
        index+=1
        precip_nc = working_dir+folder+os.path.sep+precip_file
        print(precip_nc)
        print("I am analyzing {}".format(precip_nc))
        with xr.open_dataset(precip_nc) as ds:
            print(ds)
            #print(ds['APCP_surface'].isel(latitude=114, longitude=223))
            precip = ds['APCP_surface']
            print(ds.time.attrs)
            print(np.around(hub3_wgs84_ext[1], decimals=2))
            print(np.around(hub3_wgs84_ext[0], decimals=2))
            hub3_ds = ds.sel(longitude=slice(hub3_wgs84_ext[1]-.03, 
                                             hub3_wgs84_ext[0]+.03))
            hub3_ds = hub3_ds.sel(latitude=slice(hub3_wgs84_ext[2]-.03, 
                                             hub3_wgs84_ext[3]+.03))
            #[ds.latitude >= hub3_wgs84_ext[2]]
            
            print()
            print(hub3_ds['APCP_surface'])
            print(type(hub3_ds['APCP_surface']))
            hub3_precip = hub3_ds['APCP_surface']
            hub3_avg_precip = np.average(hub3_precip)
            print(np.average(hub3_precip))
            obs_time = hub3_ds.time.item(0)#.strftime('%Y.%m.%d'))
            print(obs_time)
            date = datetime.datetime.fromtimestamp((obs_time-7200000000000) / 1e9)
            day_date = date.date()
            time_date = date.time()
            hub3_precip_dict[index] = [day_date,time_date,hub3_avg_precip]
            
            #precip.where()
            #mask by latitude longitude
        break
    break
#print(hub3_precip_dict)
hub3_precip_df = pd.DataFrame(hub3_precip_dict,
                              columns = ["day","time","avg_prec_[mm]"])

hub3_precip_df.to_csv(working_dir+"hub3_precipitation.csv")                              
print("I am done.")


In [199]:
#hub3_wgs84_file = "C:/Users/feder/Documents/CUAHSI/repos/Aquaholics_Anonymous/data/hubbard_brook/hub3/hub3_wgs84_watershed.shp"

dir_path = os.path.realpath('./')
cwd = os.getcwd()

#################################### change watershed names/folders to area of interest! #########################
#global watershed_name, watershed_foldername, watershed_nick, working_dir, coords_system
#watershed_name = "Sleeper's"
#watershed_foldername = "VT_2017_Mar_Dec"
watershed_foldername = "hubbard_brook"
watershed_nick = "hub3"
working_dir = (os.path.abspath(os.path.join(dir_path,'..')) + "/data/" + 
               watershed_foldername + os.path.sep)
print(working_dir)
hub3_wgs84_file = working_dir + watershed_nick + os.path.sep + watershed_nick+"_wgs84_watershed.shp"
hub3_wgs84_ds = driver_shp.Open(hub3_wgs84_file, 0)
hub3_wgs84_lay = hub3_wgs84_ds.GetLayer()
hub3_wgs84_ext = hub3_wgs84_lay.GetExtent()
print(hub3_wgs84_ext)

adjustment = 7200000000000
hub3_precip_dict = {}
index = 0
precip_folder = working_dir + os.path.sep +"precipitation"
precip_files = os.listdir(precip_folder)
for precip_file in precip_files:
    if  precip_file[-3:] != 'nc4':
        continue
    if '2014102602' in precip_file:
        adjustment = 3600000000000
    index+=1
    precip_nc = precip_folder+os.path.sep+precip_file
    #print(precip_nc)
    #print("I am analyzing {}".format(precip_nc))
    with xr.open_dataset(precip_nc) as ds:
        #print(ds)
        #print(ds['APCP_surface'].isel(latitude=114, longitude=223))
        precip = ds['APCP_surface']
        #print(ds.time.attrs)
        #print(np.around(hub3_wgs84_ext[1], decimals=2))
        #print(np.around(hub3_wgs84_ext[0], decimals=2))
        hub3_ds = ds.sel(longitude=slice(hub3_wgs84_ext[1]-.03, 
                                         hub3_wgs84_ext[0]+.03))
        hub3_ds = hub3_ds.sel(latitude=slice(hub3_wgs84_ext[2]-.03, 
                                         hub3_wgs84_ext[3]+.03))
        #[ds.latitude >= hub3_wgs84_ext[2]]

        #print()
        #print(hub3_ds['APCP_surface'])
        #print(type(hub3_ds['APCP_surface']))
        hub3_precip = hub3_ds['APCP_surface']
        hub3_avg_precip = np.average(hub3_precip)
        #print(np.average(hub3_precip))
        obs_time = hub3_ds.time.item(0)#.strftime('%Y.%m.%d'))
        #print(obs_time)
        date = datetime.datetime.fromtimestamp((obs_time-adjustment) / 1e9)
        day_date = str(date.date())
        time_date = str(date.time())
        print(day_date, time_date, hub3_avg_precip)
        hub3_precip_dict[index] = [day_date,time_date,hub3_avg_precip]

        #precip.where()
        #mask by latitude longitude
    #break
#print(hub3_precip_dict)
hub3_precip_df = pd.DataFrame.from_dict(hub3_precip_dict,
                                       orient = 'index',
                                       columns = ["day","time","avg_prec_[mm]"])

hub3_precip_df.to_csv(working_dir+"hub3__2014_0501_1231_precipitation.csv")                              
print("I am done.")

C:\Users\feder\Documents\CUAHSI\repos\Aquaholics_Anonymous/data/hubbard_brook\
(-71.72502931316927, -71.71653304774665, 43.95464989368979, 43.96219590116448)
2014-05-01 00:00:00 1.2
2014-05-01 01:00:00 1.725
2014-05-01 02:00:00 1.2
2014-05-01 03:00:00 2.7250001
2014-05-01 04:00:00 0.175
2014-05-01 05:00:00 0.0
2014-05-01 06:00:00 0.175
2014-05-01 07:00:00 0.175
2014-05-01 08:00:00 1.2
2014-05-01 09:00:00 2.125
2014-05-01 10:00:00 0.175
2014-05-01 11:00:00 0.175
2014-05-01 12:00:00 1.65
2014-05-01 13:00:00 0.0
2014-05-01 14:00:00 0.0
2014-05-01 15:00:00 0.0
2014-05-01 16:00:00 0.0
2014-05-01 17:00:00 0.0
2014-05-01 18:00:00 0.0
2014-05-01 19:00:00 0.0
2014-05-01 20:00:00 0.0
2014-05-01 21:00:00 0.0
2014-05-01 22:00:00 0.0
2014-05-01 23:00:00 0.0
2014-05-02 00:00:00 0.325
2014-05-02 01:00:00 0.0
2014-05-02 02:00:00 0.77500004
2014-05-02 03:00:00 0.0
2014-05-02 04:00:00 0.625
2014-05-02 05:00:00 0.0
2014-05-02 06:00:00 0.0
2014-05-02 07:00:00 0.0
2014-05-02 08:00:00 0.0
2014-05-02 09:00:0

2014-05-14 23:00:00 0.0
2014-05-15 00:00:00 0.0
2014-05-15 01:00:00 0.0
2014-05-15 02:00:00 0.0
2014-05-15 03:00:00 0.0
2014-05-15 04:00:00 0.0
2014-05-15 05:00:00 0.0
2014-05-15 06:00:00 0.0
2014-05-15 07:00:00 0.0
2014-05-15 08:00:00 0.0
2014-05-15 09:00:00 0.0
2014-05-15 10:00:00 0.0
2014-05-15 11:00:00 0.0
2014-05-15 12:00:00 0.0
2014-05-15 13:00:00 0.0
2014-05-15 14:00:00 0.0
2014-05-15 15:00:00 0.0
2014-05-15 16:00:00 0.0
2014-05-15 17:00:00 0.0
2014-05-15 18:00:00 0.0
2014-05-15 19:00:00 0.0
2014-05-15 20:00:00 0.0
2014-05-15 21:00:00 0.0
2014-05-15 22:00:00 0.0
2014-05-15 23:00:00 0.0
2014-05-16 00:00:00 0.0
2014-05-16 01:00:00 0.0
2014-05-16 02:00:00 0.0
2014-05-16 03:00:00 0.0
2014-05-16 04:00:00 0.0
2014-05-16 05:00:00 0.0
2014-05-16 06:00:00 0.0
2014-05-16 07:00:00 0.0
2014-05-16 08:00:00 0.0
2014-05-16 09:00:00 0.0
2014-05-16 10:00:00 0.0
2014-05-16 11:00:00 0.0
2014-05-16 12:00:00 0.0
2014-05-16 13:00:00 0.0
2014-05-16 14:00:00 0.0
2014-05-16 15:00:00 0.0
2014-05-16 16:00

2014-05-29 03:00:00 0.0
2014-05-29 04:00:00 0.0
2014-05-29 05:00:00 0.0
2014-05-29 06:00:00 0.0
2014-05-29 07:00:00 0.0
2014-05-29 08:00:00 0.0
2014-05-29 09:00:00 0.0
2014-05-29 10:00:00 0.0
2014-05-29 11:00:00 0.0
2014-05-29 12:00:00 0.0
2014-05-29 13:00:00 0.0
2014-05-29 14:00:00 0.0
2014-05-29 15:00:00 0.0
2014-05-29 16:00:00 0.0
2014-05-29 17:00:00 0.0
2014-05-29 18:00:00 0.0
2014-05-29 19:00:00 0.0
2014-05-29 20:00:00 0.0
2014-05-29 21:00:00 0.0
2014-05-29 22:00:00 0.0
2014-05-29 23:00:00 0.0
2014-05-30 00:00:00 0.0
2014-05-30 01:00:00 0.0
2014-05-30 02:00:00 0.0
2014-05-30 03:00:00 0.0
2014-05-30 04:00:00 0.0
2014-05-30 05:00:00 0.0
2014-05-30 06:00:00 0.0
2014-05-30 07:00:00 0.0
2014-05-30 08:00:00 0.0
2014-05-30 09:00:00 0.0
2014-05-30 10:00:00 0.0
2014-05-30 11:00:00 0.0
2014-05-30 12:00:00 0.0
2014-05-30 13:00:00 0.0
2014-05-30 14:00:00 0.0
2014-05-30 15:00:00 0.0
2014-05-30 16:00:00 0.0
2014-05-30 17:00:00 0.025
2014-05-30 18:00:00 0.25
2014-05-30 19:00:00 1.375
2014-05-30 

2014-06-12 10:00:00 0.075
2014-06-12 11:00:00 0.95000005
2014-06-12 12:00:00 1.4000001
2014-06-12 13:00:00 0.75
2014-06-12 14:00:00 0.15
2014-06-12 15:00:00 0.0
2014-06-12 16:00:00 0.0
2014-06-12 17:00:00 0.0
2014-06-12 18:00:00 0.0
2014-06-12 19:00:00 0.0
2014-06-12 20:00:00 0.0
2014-06-12 21:00:00 0.0
2014-06-12 22:00:00 0.25
2014-06-12 23:00:00 0.075
2014-06-13 00:00:00 1.375
2014-06-13 01:00:00 3.3750002
2014-06-13 02:00:00 1.325
2014-06-13 03:00:00 0.875
2014-06-13 04:00:00 2.95
2014-06-13 05:00:00 1.675
2014-06-13 06:00:00 0.975
2014-06-13 07:00:00 0.0
2014-06-13 08:00:00 0.0
2014-06-13 09:00:00 0.0
2014-06-13 10:00:00 0.0
2014-06-13 11:00:00 0.0
2014-06-13 12:00:00 0.0
2014-06-13 13:00:00 0.175
2014-06-13 14:00:00 0.0
2014-06-13 15:00:00 0.175
2014-06-13 16:00:00 0.7
2014-06-13 17:00:00 0.5
2014-06-13 18:00:00 5.8750005
2014-06-13 19:00:00 3.4499998
2014-06-13 20:00:00 1.15
2014-06-13 21:00:00 0.0
2014-06-13 22:00:00 0.0
2014-06-13 23:00:00 0.0
2014-06-14 00:00:00 0.0
2014-06-14

2014-06-27 08:00:00 0.0
2014-06-27 09:00:00 0.0
2014-06-27 10:00:00 0.0
2014-06-27 11:00:00 0.0
2014-06-27 12:00:00 0.0
2014-06-27 13:00:00 0.0
2014-06-27 14:00:00 0.0
2014-06-27 15:00:00 0.0
2014-06-27 16:00:00 0.0
2014-06-27 17:00:00 0.0
2014-06-27 18:00:00 0.0
2014-06-27 19:00:00 0.0
2014-06-27 20:00:00 0.0
2014-06-27 21:00:00 0.0
2014-06-27 22:00:00 0.0
2014-06-27 23:00:00 0.0
2014-06-28 00:00:00 0.0
2014-06-28 01:00:00 0.0
2014-06-28 02:00:00 0.0
2014-06-28 03:00:00 0.0
2014-06-28 04:00:00 0.0
2014-06-28 05:00:00 0.0
2014-06-28 06:00:00 0.0
2014-06-28 07:00:00 0.0
2014-06-28 08:00:00 0.0
2014-06-28 09:00:00 0.0
2014-06-28 10:00:00 0.0
2014-06-28 11:00:00 0.0
2014-06-28 12:00:00 0.0
2014-06-28 13:00:00 0.0
2014-06-28 14:00:00 0.0
2014-06-28 15:00:00 0.0
2014-06-28 16:00:00 0.0
2014-06-28 17:00:00 0.0
2014-06-28 18:00:00 0.0
2014-06-28 19:00:00 0.0
2014-06-28 20:00:00 0.0
2014-06-28 21:00:00 0.0
2014-06-28 22:00:00 0.0
2014-06-28 23:00:00 0.0
2014-06-29 00:00:00 0.0
2014-06-29 01:00

...

2014-10-22 18:00:00 0.0
2014-10-22 19:00:00 0.0
2014-10-22 20:00:00 0.0
2014-10-22 21:00:00 0.0
2014-10-22 22:00:00 0.0
2014-10-22 23:00:00 0.1
2014-10-23 00:00:00 0.05
2014-10-23 01:00:00 0.0
2014-10-23 02:00:00 0.0
2014-10-23 03:00:00 0.0
2014-10-23 04:00:00 0.0
2014-10-23 05:00:00 0.425
2014-10-23 06:00:00 0.82500005
2014-10-23 07:00:00 0.0
2014-10-23 08:00:00 0.0
2014-10-23 09:00:00 0.0
2014-10-23 10:00:00 0.17500001
2014-10-23 11:00:00 0.0
2014-10-23 12:00:00 0.0
2014-10-23 13:00:00 0.0
2014-10-23 14:00:00 0.7
2014-10-23 15:00:00 1.6500001
2014-10-23 16:00:00 1.625
2014-10-23 17:00:00 2.5500002
2014-10-23 18:00:00 0.2
2014-10-23 19:00:00 0.0
2014-10-23 20:00:00 2.9
2014-10-23 21:00:00 1.8750001
2014-10-23 22:00:00 0.05
2014-10-23 23:00:00 0.0
2014-10-24 00:00:00 0.0
2014-10-24 01:00:00 2.025
2014-10-24 02:00:00 0.0
2014-10-24 03:00:00 0.0
2014-10-24 04:00:00 0.1
2014-10-24 05:00:00 2.8
2014-10-24 06:00:00 3.8000002
2014-10-24 07:00:00 0.5
2014-10-24 08:00:00 0.0
2014-10-24 09:00:0

2014-11-06 12:00:00 0.0
2014-11-06 13:00:00 0.0
2014-11-06 14:00:00 0.0
2014-11-06 15:00:00 0.0
2014-11-06 16:00:00 0.0
2014-11-06 17:00:00 0.0
2014-11-06 18:00:00 0.0
2014-11-06 19:00:00 0.0
2014-11-06 20:00:00 0.0
2014-11-06 21:00:00 0.275
2014-11-06 22:00:00 0.125
2014-11-06 23:00:00 0.275
2014-11-07 00:00:00 0.15
2014-11-07 01:00:00 0.2
2014-11-07 02:00:00 0.35000002
2014-11-07 03:00:00 0.0
2014-11-07 04:00:00 0.77500004
2014-11-07 05:00:00 0.95000005
2014-11-07 06:00:00 1.4000001
2014-11-07 07:00:00 0.15
2014-11-07 08:00:00 0.0
2014-11-07 09:00:00 0.0
2014-11-07 10:00:00 0.0
2014-11-07 11:00:00 0.0
2014-11-07 12:00:00 0.15
2014-11-07 13:00:00 0.0
2014-11-07 14:00:00 0.0
2014-11-07 15:00:00 0.0
2014-11-07 16:00:00 0.0
2014-11-07 17:00:00 0.075
2014-11-07 18:00:00 0.0
2014-11-07 19:00:00 0.0
2014-11-07 20:00:00 0.0
2014-11-07 21:00:00 0.025
2014-11-07 22:00:00 0.0
2014-11-07 23:00:00 0.0
2014-11-08 00:00:00 0.0
2014-11-08 01:00:00 0.0
2014-11-08 02:00:00 0.0
2014-11-08 03:00:00 0.0


2014-11-21 02:00:00 0.0
2014-11-21 03:00:00 0.0
2014-11-21 04:00:00 0.0
2014-11-21 05:00:00 0.0
2014-11-21 06:00:00 0.0
2014-11-21 07:00:00 0.0
2014-11-21 08:00:00 0.0
2014-11-21 09:00:00 0.0
2014-11-21 10:00:00 0.0
2014-11-21 11:00:00 0.0
2014-11-21 12:00:00 0.0
2014-11-21 13:00:00 0.0
2014-11-21 14:00:00 0.0
2014-11-21 15:00:00 0.0
2014-11-21 16:00:00 0.0
2014-11-21 17:00:00 0.0
2014-11-21 18:00:00 0.0
2014-11-21 19:00:00 0.0
2014-11-21 20:00:00 0.0
2014-11-21 21:00:00 0.0
2014-11-21 22:00:00 0.0
2014-11-21 23:00:00 0.0
2014-11-22 00:00:00 0.0
2014-11-22 01:00:00 0.0
2014-11-22 02:00:00 0.0
2014-11-22 03:00:00 0.0
2014-11-22 04:00:00 0.0
2014-11-22 05:00:00 0.0
2014-11-22 06:00:00 0.0
2014-11-22 07:00:00 0.0
2014-11-22 08:00:00 0.0
2014-11-22 09:00:00 0.0
2014-11-22 10:00:00 0.0
2014-11-22 11:00:00 0.0
2014-11-22 12:00:00 0.0
2014-11-22 13:00:00 0.0
2014-11-22 14:00:00 0.0
2014-11-22 15:00:00 0.0
2014-11-22 16:00:00 0.0
2014-11-22 17:00:00 0.0
2014-11-22 18:00:00 0.0
2014-11-22 19:00

2014-12-05 19:00:00 0.0
2014-12-05 20:00:00 0.0
2014-12-05 21:00:00 0.0
2014-12-05 22:00:00 0.0
2014-12-05 23:00:00 0.0
2014-12-06 00:00:00 0.0
2014-12-06 01:00:00 0.0
2014-12-06 02:00:00 0.1
2014-12-06 03:00:00 0.375
2014-12-06 04:00:00 1.15
2014-12-06 05:00:00 1.9750001
2014-12-06 06:00:00 2.2000003
2014-12-06 07:00:00 2.05
2014-12-06 08:00:00 2.35
2014-12-06 09:00:00 1.225
2014-12-06 10:00:00 1.0500001
2014-12-06 11:00:00 1.0250001
2014-12-06 12:00:00 0.47500002
2014-12-06 13:00:00 0.35000002
2014-12-06 14:00:00 1.125
2014-12-06 15:00:00 2.625
2014-12-06 16:00:00 4.125
2014-12-06 17:00:00 3.15
2014-12-06 18:00:00 1.4
2014-12-06 19:00:00 1.0500001
2014-12-06 20:00:00 1.075
2014-12-06 21:00:00 0.4
2014-12-06 22:00:00 0.475
2014-12-06 23:00:00 0.225
2014-12-07 00:00:00 0.0
2014-12-07 01:00:00 0.275
2014-12-07 02:00:00 0.0
2014-12-07 03:00:00 0.0
2014-12-07 04:00:00 0.0
2014-12-07 05:00:00 0.0
2014-12-07 06:00:00 0.0
2014-12-07 07:00:00 0.0
2014-12-07 08:00:00 0.0
2014-12-07 09:00:00 0.

2014-12-20 10:00:00 0.0
2014-12-20 11:00:00 0.0
2014-12-20 12:00:00 0.0
2014-12-20 13:00:00 0.0
2014-12-20 14:00:00 0.0
2014-12-20 15:00:00 0.0
2014-12-20 16:00:00 0.0
2014-12-20 17:00:00 0.0
2014-12-20 18:00:00 0.0
2014-12-20 19:00:00 0.0
2014-12-20 20:00:00 0.0
2014-12-20 21:00:00 0.0
2014-12-20 22:00:00 0.0
2014-12-20 23:00:00 0.0
2014-12-21 00:00:00 0.0
2014-12-21 01:00:00 0.0
2014-12-21 02:00:00 0.0
2014-12-21 03:00:00 0.0
2014-12-21 04:00:00 0.0
2014-12-21 05:00:00 0.0
2014-12-21 06:00:00 0.0
2014-12-21 07:00:00 0.0
2014-12-21 08:00:00 0.0
2014-12-21 09:00:00 0.0
2014-12-21 10:00:00 0.0
2014-12-21 11:00:00 0.0
2014-12-21 12:00:00 0.0
2014-12-21 13:00:00 0.0
2014-12-21 14:00:00 0.0
2014-12-21 15:00:00 0.0
2014-12-21 16:00:00 0.075
2014-12-21 17:00:00 0.0
2014-12-21 18:00:00 0.0
2014-12-21 19:00:00 0.0
2014-12-21 20:00:00 0.0
2014-12-21 21:00:00 0.0
2014-12-21 22:00:00 0.0
2014-12-21 23:00:00 0.0
2014-12-22 00:00:00 0.0
2014-12-22 01:00:00 0.0
2014-12-22 02:00:00 0.0
2014-12-22 03:

In [200]:
#hub7_wgs84_file = "C:/Users/feder/Documents/CUAHSI/repos/Aquaholics_Anonymous/data/hubbard_brook/hub7/hub7_wgs84_watershed.shp"

dir_path = os.path.realpath('./')
cwd = os.getcwd()

#################################### change watershed names/folders to area of interest! #########################
#global watershed_name, watershed_foldername, watershed_nick, working_dir, coords_system
#watershed_name = "Sleeper's"
#watershed_foldername = "VT_2017_Mar_Dec"
watershed_foldername = "hubbard_brook"
watershed_nick = "hub7"
working_dir = (os.path.abspath(os.path.join(dir_path,'..')) + "/data/" + 
               watershed_foldername + os.path.sep)
print(working_dir)
hub7_wgs84_file = working_dir + watershed_nick + os.path.sep + watershed_nick+"_wgs84_watershed.shp"
hub7_wgs84_ds = driver_shp.Open(hub7_wgs84_file, 0)
hub7_wgs84_lay = hub7_wgs84_ds.GetLayer()
hub7_wgs84_ext = hub7_wgs84_lay.GetExtent()
print(hub7_wgs84_ext)

adjustment = 7200000000000
hub7_precip_dict = {}
index = 0
precip_folder = working_dir + os.path.sep +"precipitation"
precip_files = os.listdir(precip_folder)
for precip_file in precip_files:
    if  precip_file[-3:] != 'nc4':
        continue
    if '2014102602' in precip_file:
        adjustment = 3600000000000
    index+=1
    precip_nc = precip_folder+os.path.sep+precip_file
    #print(precip_nc)
    #print("I am analyzing {}".format(precip_nc))
    with xr.open_dataset(precip_nc) as ds:
        precip = ds['APCP_surface']
        hub7_ds = ds.sel(longitude=slice(hub7_wgs84_ext[1]-.03, 
                                         hub7_wgs84_ext[0]+.03))
        hub7_ds = hub7_ds.sel(latitude=slice(hub7_wgs84_ext[2]-.03, 
                                         hub7_wgs84_ext[3]+.03))
        #[ds.latitude >= hub7_wgs84_ext[2]]

        #print()
        #print(hub7_ds['APCP_surface'])
        #print(type(hub7_ds['APCP_surface']))
        hub7_precip = hub7_ds['APCP_surface']
        hub7_avg_precip = np.average(hub7_precip)
        #print(np.average(hub7_precip))
        obs_time = hub7_ds.time.item(0)#.strftime('%Y.%m.%d'))
        #print(obs_time)
        date = datetime.datetime.fromtimestamp((obs_time - adjustment) / 1e9)
        day_date = str(date.date())
        time_date = str(date.time())
        print(day_date, time_date, hub7_avg_precip)
        hub7_precip_dict[index] = [day_date,time_date,hub7_avg_precip]

        #precip.where()
        #mask by latitude longitude
    #break
#print(hub7_precip_dict)
hub7_precip_df = pd.DataFrame.from_dict(hub7_precip_dict,
                                       orient = 'index',
                                       columns = ["day","time","avg_prec_[mm]"])

hub7_precip_df.to_csv(working_dir+"hub7__2014_0501_1231_precipitation.csv")                              
print("I am done.")

C:\Users\feder\Documents\CUAHSI\repos\Aquaholics_Anonymous/data/hubbard_brook\
(-71.77298933727646, -71.75834370381978, 43.916086098978916, 43.927562620275445)
2014-05-01 00:00:00 1.1500001
2014-05-01 01:00:00 2.1
2014-05-01 02:00:00 1.4000001
2014-05-01 03:00:00 2.55
2014-05-01 04:00:00 0.2
2014-05-01 05:00:00 0.0
2014-05-01 06:00:00 0.2
2014-05-01 07:00:00 0.2
2014-05-01 08:00:00 1.6
2014-05-01 09:00:00 1.95
2014-05-01 10:00:00 0.2
2014-05-01 11:00:00 0.2
2014-05-01 12:00:00 2.1
2014-05-01 13:00:00 0.0
2014-05-01 14:00:00 0.0
2014-05-01 15:00:00 0.0
2014-05-01 16:00:00 0.0
2014-05-01 17:00:00 0.0
2014-05-01 18:00:00 0.0
2014-05-01 19:00:00 0.0
2014-05-01 20:00:00 0.0
2014-05-01 21:00:00 0.0
2014-05-01 22:00:00 0.0
2014-05-01 23:00:00 0.0
2014-05-02 00:00:00 0.35
2014-05-02 01:00:00 0.0
2014-05-02 02:00:00 0.7
2014-05-02 03:00:00 0.0
2014-05-02 04:00:00 0.6
2014-05-02 05:00:00 0.0
2014-05-02 06:00:00 0.0
2014-05-02 07:00:00 0.0
2014-05-02 08:00:00 0.0
2014-05-02 09:00:00 0.0
2014-05-0

2014-05-15 13:00:00 0.0
2014-05-15 14:00:00 0.0
2014-05-15 15:00:00 0.0
2014-05-15 16:00:00 0.0
2014-05-15 17:00:00 0.0
2014-05-15 18:00:00 0.0
2014-05-15 19:00:00 0.0
2014-05-15 20:00:00 0.0
2014-05-15 21:00:00 0.0
2014-05-15 22:00:00 0.0
2014-05-15 23:00:00 0.0
2014-05-16 00:00:00 0.0
2014-05-16 01:00:00 0.0
2014-05-16 02:00:00 0.0
2014-05-16 03:00:00 0.0
2014-05-16 04:00:00 0.0
2014-05-16 05:00:00 0.0
2014-05-16 06:00:00 0.0
2014-05-16 07:00:00 0.0
2014-05-16 08:00:00 0.0
2014-05-16 09:00:00 0.0
2014-05-16 10:00:00 0.0
2014-05-16 11:00:00 0.0
2014-05-16 12:00:00 0.0
2014-05-16 13:00:00 0.0
2014-05-16 14:00:00 0.0
2014-05-16 15:00:00 0.0
2014-05-16 16:00:00 0.0
2014-05-16 17:00:00 0.0
2014-05-16 18:00:00 0.0
2014-05-16 19:00:00 0.15
2014-05-16 20:00:00 0.2
2014-05-16 21:00:00 0.0
2014-05-16 22:00:00 0.0
2014-05-16 23:00:00 0.0
2014-05-17 00:00:00 0.0
2014-05-17 01:00:00 0.0
2014-05-17 02:00:00 1.5
2014-05-17 03:00:00 0.0
2014-05-17 04:00:00 0.2
2014-05-17 05:00:00 0.2
2014-05-17 06:0

2014-05-30 02:00:00 0.0
2014-05-30 03:00:00 0.0
2014-05-30 04:00:00 0.0
2014-05-30 05:00:00 0.0
2014-05-30 06:00:00 0.0
2014-05-30 07:00:00 0.0
2014-05-30 08:00:00 0.0
2014-05-30 09:00:00 0.0
2014-05-30 10:00:00 0.0
2014-05-30 11:00:00 0.0
2014-05-30 12:00:00 0.0
2014-05-30 13:00:00 0.0
2014-05-30 14:00:00 0.0
2014-05-30 15:00:00 0.0
2014-05-30 16:00:00 0.0
2014-05-30 17:00:00 0.3
2014-05-30 18:00:00 1.3
2014-05-30 19:00:00 0.55
2014-05-30 20:00:00 1.2
2014-05-30 21:00:00 3.6
2014-05-30 22:00:00 2.35
2014-05-30 23:00:00 0.0
2014-05-31 00:00:00 0.0
2014-05-31 01:00:00 0.0
2014-05-31 02:00:00 0.0
2014-05-31 03:00:00 0.0
2014-05-31 04:00:00 0.0
2014-05-31 05:00:00 0.0
2014-05-31 06:00:00 0.0
2014-05-31 07:00:00 0.0
2014-05-31 08:00:00 0.0
2014-05-31 09:00:00 0.0
2014-05-31 10:00:00 0.0
2014-05-31 11:00:00 0.0
2014-05-31 12:00:00 0.0
2014-05-31 13:00:00 0.0
2014-05-31 14:00:00 0.0
2014-05-31 15:00:00 0.0
2014-05-31 16:00:00 0.0
2014-05-31 17:00:00 0.0
2014-05-31 18:00:00 0.0
2014-05-31 19:

2014-06-14 03:00:00 0.05
2014-06-14 04:00:00 0.0
2014-06-14 05:00:00 0.0
2014-06-14 06:00:00 0.0
2014-06-14 07:00:00 0.0
2014-06-14 08:00:00 0.8
2014-06-14 09:00:00 0.15
2014-06-14 10:00:00 0.0
2014-06-14 11:00:00 0.0
2014-06-14 12:00:00 0.0
2014-06-14 13:00:00 0.0
2014-06-14 14:00:00 0.0
2014-06-14 15:00:00 0.0
2014-06-14 16:00:00 0.0
2014-06-14 17:00:00 0.0
2014-06-14 18:00:00 0.0
2014-06-14 19:00:00 0.0
2014-06-14 20:00:00 0.0
2014-06-14 21:00:00 0.0
2014-06-14 22:00:00 0.0
2014-06-14 23:00:00 0.0
2014-06-15 00:00:00 0.0
2014-06-15 01:00:00 0.0
2014-06-15 02:00:00 0.0
2014-06-15 03:00:00 0.0
2014-06-15 04:00:00 0.0
2014-06-15 05:00:00 0.0
2014-06-15 06:00:00 0.0
2014-06-15 07:00:00 0.0
2014-06-15 08:00:00 0.0
2014-06-15 09:00:00 0.0
2014-06-15 10:00:00 0.0
2014-06-15 11:00:00 0.0
2014-06-15 12:00:00 0.0
2014-06-15 13:00:00 0.0
2014-06-15 14:00:00 0.0
2014-06-15 15:00:00 0.0
2014-06-15 16:00:00 0.0
2014-06-15 17:00:00 0.0
2014-06-15 18:00:00 0.0
2014-06-15 19:00:00 0.0
2014-06-15 20:

2014-06-28 16:00:00 0.0
2014-06-28 17:00:00 0.0
2014-06-28 18:00:00 0.0
2014-06-28 19:00:00 0.0
2014-06-28 20:00:00 0.0
2014-06-28 21:00:00 0.0
2014-06-28 22:00:00 0.0
2014-06-28 23:00:00 0.0
2014-06-29 00:00:00 0.0
2014-06-29 01:00:00 0.0
2014-06-29 02:00:00 0.0
2014-06-29 03:00:00 0.0
2014-06-29 04:00:00 0.0
2014-06-29 05:00:00 0.0
2014-06-29 06:00:00 0.0
2014-06-29 07:00:00 0.0
2014-06-29 08:00:00 0.0
2014-06-29 09:00:00 0.0
2014-06-29 10:00:00 0.0
2014-06-29 11:00:00 0.0
2014-06-29 12:00:00 0.0
2014-06-29 13:00:00 0.0
2014-06-29 14:00:00 0.0
2014-06-29 15:00:00 0.0
2014-06-29 16:00:00 0.0
2014-06-29 17:00:00 0.0
2014-06-29 18:00:00 0.0
2014-06-29 19:00:00 0.0
2014-06-29 20:00:00 0.0
2014-06-29 21:00:00 0.0
2014-06-29 22:00:00 0.0
2014-06-29 23:00:00 0.0
2014-06-30 00:00:00 0.0
2014-06-30 01:00:00 0.0
2014-06-30 02:00:00 0.0
2014-06-30 03:00:00 0.0
2014-06-30 04:00:00 0.0
2014-06-30 05:00:00 0.0
2014-06-30 06:00:00 0.0
2014-06-30 07:00:00 0.0
2014-06-30 08:00:00 0.0
2014-06-30 09:00

...

2014-10-23 14:00:00 1.95
2014-10-23 15:00:00 2.3000002
2014-10-23 16:00:00 2.6
2014-10-23 17:00:00 3.65
2014-10-23 18:00:00 0.4
2014-10-23 19:00:00 0.2
2014-10-23 20:00:00 1.75
2014-10-23 21:00:00 2.15
2014-10-23 22:00:00 0.1
2014-10-23 23:00:00 0.0
2014-10-24 00:00:00 0.0
2014-10-24 01:00:00 2.35
2014-10-24 02:00:00 0.0
2014-10-24 03:00:00 0.0
2014-10-24 04:00:00 0.0
2014-10-24 05:00:00 2.95
2014-10-24 06:00:00 3.65
2014-10-24 07:00:00 0.55
2014-10-24 08:00:00 0.0
2014-10-24 09:00:00 0.0
2014-10-24 10:00:00 0.1
2014-10-24 11:00:00 0.0
2014-10-24 12:00:00 0.1
2014-10-24 13:00:00 0.0
2014-10-24 14:00:00 0.1
2014-10-24 15:00:00 0.0
2014-10-24 16:00:00 0.25
2014-10-24 17:00:00 0.2
2014-10-24 18:00:00 0.0
2014-10-24 19:00:00 0.0
2014-10-24 20:00:00 0.0
2014-10-24 21:00:00 0.0
2014-10-24 22:00:00 0.0
2014-10-24 23:00:00 0.0
2014-10-25 00:00:00 0.0
2014-10-25 01:00:00 0.0
2014-10-25 02:00:00 0.0
2014-10-25 03:00:00 0.0
2014-10-25 04:00:00 0.0
2014-10-25 05:00:00 0.0
2014-10-25 06:00:00 0.0
2

2014-11-07 04:00:00 0.90000004
2014-11-07 05:00:00 1.05
2014-11-07 06:00:00 1.2
2014-11-07 07:00:00 0.1
2014-11-07 08:00:00 0.0
2014-11-07 09:00:00 0.0
2014-11-07 10:00:00 0.0
2014-11-07 11:00:00 0.0
2014-11-07 12:00:00 0.1
2014-11-07 13:00:00 0.0
2014-11-07 14:00:00 0.0
2014-11-07 15:00:00 0.0
2014-11-07 16:00:00 0.0
2014-11-07 17:00:00 0.0
2014-11-07 18:00:00 0.0
2014-11-07 19:00:00 0.0
2014-11-07 20:00:00 0.0
2014-11-07 21:00:00 0.0
2014-11-07 22:00:00 0.0
2014-11-07 23:00:00 0.0
2014-11-08 00:00:00 0.0
2014-11-08 01:00:00 0.0
2014-11-08 02:00:00 0.0
2014-11-08 03:00:00 0.0
2014-11-08 04:00:00 0.0
2014-11-08 05:00:00 0.0
2014-11-08 06:00:00 0.0
2014-11-08 07:00:00 0.0
2014-11-08 08:00:00 0.0
2014-11-08 09:00:00 0.0
2014-11-08 10:00:00 0.0
2014-11-08 11:00:00 0.0
2014-11-08 12:00:00 0.0
2014-11-08 13:00:00 0.0
2014-11-08 14:00:00 0.0
2014-11-08 15:00:00 0.0
2014-11-08 16:00:00 0.0
2014-11-08 17:00:00 0.0
2014-11-08 18:00:00 0.0
2014-11-08 19:00:00 0.0
2014-11-08 20:00:00 0.0
2014-11-

2014-11-22 06:00:00 0.0
2014-11-22 07:00:00 0.0
2014-11-22 08:00:00 0.0
2014-11-22 09:00:00 0.0
2014-11-22 10:00:00 0.0
2014-11-22 11:00:00 0.0
2014-11-22 12:00:00 0.0
2014-11-22 13:00:00 0.0
2014-11-22 14:00:00 0.0
2014-11-22 15:00:00 0.0
2014-11-22 16:00:00 0.0
2014-11-22 17:00:00 0.0
2014-11-22 18:00:00 0.0
2014-11-22 19:00:00 0.0
2014-11-22 20:00:00 0.0
2014-11-22 21:00:00 0.0
2014-11-22 22:00:00 0.0
2014-11-22 23:00:00 0.2
2014-11-23 00:00:00 0.0
2014-11-23 01:00:00 0.0
2014-11-23 02:00:00 0.0
2014-11-23 03:00:00 0.0
2014-11-23 04:00:00 0.0
2014-11-23 05:00:00 0.0
2014-11-23 06:00:00 0.0
2014-11-23 07:00:00 0.0
2014-11-23 08:00:00 1.2
2014-11-23 09:00:00 3.75
2014-11-23 10:00:00 0.0
2014-11-23 11:00:00 0.0
2014-11-23 12:00:00 0.0
2014-11-23 13:00:00 0.0
2014-11-23 14:00:00 0.0
2014-11-23 15:00:00 0.0
2014-11-23 16:00:00 0.0
2014-11-23 17:00:00 0.0
2014-11-23 18:00:00 0.0
2014-11-23 19:00:00 0.0
2014-11-23 20:00:00 0.0
2014-11-23 21:00:00 0.0
2014-11-23 22:00:00 0.0
2014-11-23 23:0

2014-12-06 13:00:00 0.4
2014-12-06 14:00:00 1.25
2014-12-06 15:00:00 2.95
2014-12-06 16:00:00 4.8
2014-12-06 17:00:00 3.5
2014-12-06 18:00:00 1.4000001
2014-12-06 19:00:00 1.2
2014-12-06 20:00:00 1.2
2014-12-06 21:00:00 0.45
2014-12-06 22:00:00 0.5
2014-12-06 23:00:00 0.2
2014-12-07 00:00:00 0.1
2014-12-07 01:00:00 0.35000002
2014-12-07 02:00:00 0.0
2014-12-07 03:00:00 0.0
2014-12-07 04:00:00 0.0
2014-12-07 05:00:00 0.0
2014-12-07 06:00:00 0.0
2014-12-07 07:00:00 0.0
2014-12-07 08:00:00 0.0
2014-12-07 09:00:00 0.0
2014-12-07 10:00:00 0.0
2014-12-07 11:00:00 0.0
2014-12-07 12:00:00 0.0
2014-12-07 13:00:00 0.0
2014-12-07 14:00:00 0.0
2014-12-07 15:00:00 0.0
2014-12-07 16:00:00 0.0
2014-12-07 17:00:00 0.0
2014-12-07 18:00:00 0.0
2014-12-07 19:00:00 0.0
2014-12-07 20:00:00 0.0
2014-12-07 21:00:00 0.0
2014-12-07 22:00:00 0.0
2014-12-07 23:00:00 0.0
2014-12-08 00:00:00 0.0
2014-12-08 01:00:00 0.0
2014-12-08 02:00:00 0.0
2014-12-08 03:00:00 0.0
2014-12-08 04:00:00 0.0
2014-12-08 05:00:00 0.0


2014-12-21 12:00:00 0.0
2014-12-21 13:00:00 0.0
2014-12-21 14:00:00 0.0
2014-12-21 15:00:00 0.0
2014-12-21 16:00:00 0.0
2014-12-21 17:00:00 0.0
2014-12-21 18:00:00 0.0
2014-12-21 19:00:00 0.0
2014-12-21 20:00:00 0.0
2014-12-21 21:00:00 0.0
2014-12-21 22:00:00 0.0
2014-12-21 23:00:00 0.0
2014-12-22 00:00:00 0.0
2014-12-22 01:00:00 0.0
2014-12-22 02:00:00 0.0
2014-12-22 03:00:00 0.0
2014-12-22 04:00:00 0.0
2014-12-22 05:00:00 0.0
2014-12-22 06:00:00 0.0
2014-12-22 07:00:00 0.0
2014-12-22 08:00:00 0.0
2014-12-22 09:00:00 0.0
2014-12-22 10:00:00 0.0
2014-12-22 11:00:00 0.0
2014-12-22 12:00:00 0.0
2014-12-22 13:00:00 0.0
2014-12-22 14:00:00 0.0
2014-12-22 15:00:00 0.0
2014-12-22 16:00:00 0.0
2014-12-22 17:00:00 0.0
2014-12-22 18:00:00 0.0
2014-12-22 19:00:00 0.0
2014-12-22 20:00:00 0.0
2014-12-22 21:00:00 0.0
2014-12-22 22:00:00 0.0
2014-12-22 23:00:00 0.0
2014-12-23 00:00:00 0.0
2014-12-23 01:00:00 0.0
2014-12-23 02:00:00 0.0
2014-12-23 03:00:00 0.0
2014-12-23 04:00:00 0.0
2014-12-23 05:00

In [198]:
#hub7_wgs84_file = "C:/Users/feder/Documents/CUAHSI/repos/Aquaholics_Anonymous/data/hubbard_brook/hub7/hub7_wgs84_watershed.shp"

dir_path = os.path.realpath('./')
cwd = os.getcwd()

#################################### change watershed names/folders to area of interest! #########################
#global watershed_name, watershed_foldername, watershed_nick, working_dir, coords_system
#watershed_name = "Sleeper's"
#watershed_foldername = "VT_2017_Mar_Dec"
watershed_foldername = "hubbard_brook"
watershed_nick = "hub7"
working_dir = (os.path.abspath(os.path.join(dir_path,'..')) + "/data/" + 
               watershed_foldername + os.path.sep)
print(working_dir)
hub7_wgs84_file = working_dir + watershed_nick + os.path.sep + watershed_nick+"_wgs84_watershed.shp"
hub7_wgs84_ds = driver_shp.Open(hub7_wgs84_file, 0)
hub7_wgs84_lay = hub7_wgs84_ds.GetLayer()
hub7_wgs84_ext = hub7_wgs84_lay.GetExtent()
print(hub7_wgs84_ext)

adjustment = 7200000000000
hub7_precip_dict = {}
index = 0
precip_folder = working_dir + os.path.sep +"precipitation"
precip_files = os.listdir(precip_folder)
for precip_file in precip_files:
    if  precip_file[-3:] != 'nc4' or '20141026' not in precip_file:
        continue
    if '2014102602' in precip_file:
        adjustment = 3600000000000
    index+=1
    precip_nc = precip_folder+os.path.sep+precip_file
    #print(precip_nc)
    #print("I am analyzing {}".format(precip_nc))
    with xr.open_dataset(precip_nc) as ds:
        precip = ds['APCP_surface']
        hub7_ds = ds.sel(longitude=slice(hub7_wgs84_ext[1]-.03, 
                                         hub7_wgs84_ext[0]+.03))
        hub7_ds = hub7_ds.sel(latitude=slice(hub7_wgs84_ext[2]-.03, 
                                         hub7_wgs84_ext[3]+.03))
        #[ds.latitude >= hub7_wgs84_ext[2]]

        #print()
        print(hub7_ds['APCP_surface'])
        #print(type(hub7_ds['APCP_surface']))
        hub7_precip = hub7_ds['APCP_surface']
        hub7_avg_precip = np.average(hub7_precip)
        #print(np.average(hub7_precip))
        obs_time = hub7_ds.time.item(0)#.strftime('%Y.%m.%d'))
        print(obs_time)
        date = datetime.datetime.fromtimestamp((obs_time-adjustment) / 1e9)
        day_date = str(date.date())
        time_date = str(date.time())
        print(day_date, time_date, hub7_avg_precip)
        print('\n')
        hub7_precip_dict[index] = [day_date,time_date,hub7_avg_precip]

        #precip.where()
        #mask by latitude longitude
    #break
#print(hub7_precip_dict)
hub7_precip_df = pd.DataFrame.from_dict(hub7_precip_dict,
                                       orient = 'index',
                                       columns = ["day","time","avg_prec_[mm]"])

#hub7_precip_df.to_csv(working_dir+"hub7__2014_0501_1231_precipitation.csv")                              
print("I am done.")

C:\Users\feder\Documents\CUAHSI\repos\Aquaholics_Anonymous/data/hubbard_brook\
(-71.77298933727646, -71.75834370381978, 43.916086098978916, 43.927562620275445)
<xarray.DataArray 'APCP_surface' (time: 1, latitude: 2, longitude: 1)>
array([[[0.],
        [0.]]], dtype=float32)
Coordinates:
  * latitude   (latitude) float64 43.9 43.93
  * longitude  (longitude) float64 -71.77
  * time       (time) datetime64[ns] 2014-10-26
Attributes:
    long_name:   Total Precipitation
    units:       kg/m^2
    level:       surface
    short_name:  APCP_surface
1414281600000000000
2014-10-26 00:00:00 0.0


<xarray.DataArray 'APCP_surface' (time: 1, latitude: 2, longitude: 1)>
array([[[0.5],
        [0.8]]], dtype=float32)
Coordinates:
  * latitude   (latitude) float64 43.9 43.93
  * longitude  (longitude) float64 -71.77
  * time       (time) datetime64[ns] 2014-10-26T01:00:00
Attributes:
    long_name:   Total Precipitation
    units:       kg/m^2
    level:       surface
    short_name:  APCP_surface

In [220]:
dir_path = os.path.realpath('./')
cwd = os.getcwd()

#################################### change watershed names/folders to area of interest! #########################
#global watershed_name, watershed_foldername, watershed_nick, working_dir, coords_system
#watershed_name = "Sleeper's"
#watershed_foldername = "VT_2017_Mar_Dec"
watershed_foldername = "sleepers"
watershed_nick = "slee"
working_dir = (os.path.abspath(os.path.join(dir_path,'..')) + "/data/" + 
               watershed_foldername + os.path.sep)
print(working_dir)
slee_wgs84_file = working_dir + "w_3_wgs84_projected.shp"
slee_wgs84_ds = driver_shp.Open(slee_wgs84_file, 0)
slee_wgs84_lay = slee_wgs84_ds.GetLayer()
slee_wgs84_ext = slee_wgs84_lay.GetExtent()
print(slee_wgs84_ext)

adjustment = 7200000000000
slee_precip_dict = {}
index = 0
precip_folder = working_dir + os.path.sep +"precipitation"
precip_files = os.listdir(precip_folder)
for precip_file in precip_files:
    if  precip_file[-3:] != 'nc4':# or '20171029' not in precip_file:
        continue
    if '2017102902' in precip_file:
        adjustment = 3600000000000
    index+=1
    precip_nc = precip_folder+os.path.sep+precip_file
    #print(precip_nc)
    #print("I am analyzing {}".format(precip_nc))
    with xr.open_dataset(precip_nc) as ds:
        precip = ds['APCP_surface']
        slee_ds = ds.sel(longitude=slice(slee_wgs84_ext[1]-.05, 
                                         slee_wgs84_ext[0]+.05))
        slee_ds = slee_ds.sel(latitude=slice(slee_wgs84_ext[2]-.03, 
                                         slee_wgs84_ext[3]+.03))
        #[ds.latitude >= hub7_wgs84_ext[2]]

        #print()
        #print(slee_ds['APCP_surface'])
        #print(type(slee_ds['APCP_surface']))
        slee_precip = slee_ds['APCP_surface']
        slee_avg_precip = np.average(slee_precip)
        #print(np.average(hub7_precip))
        obs_time = slee_ds.time.item(0)#.strftime('%Y.%m.%d'))
        #print(obs_time)
        date = datetime.datetime.fromtimestamp((obs_time - adjustment) / 1e9)
        day_date = str(date.date())
        time_date = str(date.time())
        print(day_date, time_date, slee_avg_precip)
        #print('\n')
        slee_precip_dict[index] = [day_date,time_date,slee_avg_precip]

        #precip.where()
        #mask by latitude longitude
    #break
#print(slee_precip_dict)
slee_precip_df = pd.DataFrame.from_dict(slee_precip_dict,
                                       orient = 'index',
                                       columns = ["day","time","avg_prec_[mm]"])

slee_precip_df.to_csv(working_dir+"slee__2017_0401_1228_precipitation.csv")                              
print("I am done.")

C:\Users\feder\Documents\CUAHSI\repos\Aquaholics_Anonymous/data/sleepers\
(-72.17742194495885, -72.12086182551258, 44.47318804495044, 44.50045587536506)
2017-04-01 00:00:00 0.87500006


2017-04-01 01:00:00 0.75


2017-04-01 02:00:00 1.025


2017-04-01 03:00:00 1.625


2017-04-01 04:00:00 1.525


2017-04-01 05:00:00 1.55


2017-04-01 06:00:00 1.1000001


2017-04-01 07:00:00 1.5250001


2017-04-01 08:00:00 1.25


2017-04-01 09:00:00 0.475


2017-04-01 10:00:00 0.075


2017-04-01 11:00:00 0.475


2017-04-01 12:00:00 0.35


2017-04-01 13:00:00 0.85


2017-04-01 14:00:00 0.72499996


2017-04-01 15:00:00 0.975


2017-04-01 16:00:00 0.55


2017-04-01 17:00:00 0.375


2017-04-01 18:00:00 0.32500002


2017-04-01 19:00:00 0.2


2017-04-01 20:00:00 0.125


2017-04-01 21:00:00 0.0


2017-04-01 22:00:00 0.0


2017-04-01 23:00:00 0.075


2017-04-02 00:00:00 0.0


2017-04-02 01:00:00 0.0


2017-04-02 02:00:00 0.0


2017-04-02 03:00:00 0.0


2017-04-02 04:00:00 0.0


2017-04-02 05:00:00 0.0


2017-04-

2017-04-14 07:00:00 0.0


2017-04-14 08:00:00 0.0


2017-04-14 09:00:00 0.0


2017-04-14 10:00:00 0.0


2017-04-14 11:00:00 0.0


2017-04-14 12:00:00 0.0


2017-04-14 13:00:00 0.0


2017-04-14 14:00:00 0.0


2017-04-14 15:00:00 0.0


2017-04-14 16:00:00 0.0


2017-04-14 17:00:00 0.0


2017-04-14 18:00:00 0.0


2017-04-14 19:00:00 0.0


2017-04-14 20:00:00 0.0


2017-04-14 21:00:00 0.0


2017-04-14 22:00:00 0.0


2017-04-14 23:00:00 0.0


2017-04-15 00:00:00 0.0


2017-04-15 01:00:00 0.0


2017-04-15 02:00:00 0.0


2017-04-15 03:00:00 0.0


2017-04-15 04:00:00 0.0


2017-04-15 05:00:00 0.0


2017-04-15 06:00:00 0.0


2017-04-15 07:00:00 0.0


2017-04-15 08:00:00 0.0


2017-04-15 09:00:00 0.0


2017-04-15 10:00:00 0.0


2017-04-15 11:00:00 0.0


2017-04-15 12:00:00 0.0


2017-04-15 13:00:00 0.0


2017-04-15 14:00:00 0.0


2017-04-15 15:00:00 0.0


2017-04-15 16:00:00 0.0


2017-04-15 17:00:00 0.0


2017-04-15 18:00:00 0.0


2017-04-15 19:00:00 0.0


2017-04-15 20:00:00 0.0


2017-04-15 2

2017-04-27 16:00:00 0.0


2017-04-27 17:00:00 0.0


2017-04-27 18:00:00 0.0


2017-04-27 19:00:00 0.0


2017-04-27 20:00:00 0.0


2017-04-27 21:00:00 0.0


2017-04-27 22:00:00 0.0


2017-04-27 23:00:00 0.0


2017-04-28 00:00:00 0.0


2017-04-28 01:00:00 0.0


2017-04-28 02:00:00 0.0


2017-04-28 03:00:00 0.0


2017-04-28 04:00:00 0.0


2017-04-28 05:00:00 0.0


2017-04-28 06:00:00 0.0


2017-04-28 07:00:00 0.0


2017-04-28 08:00:00 0.0


2017-04-28 09:00:00 0.0


2017-04-28 10:00:00 0.0


2017-04-28 11:00:00 0.0


2017-04-28 12:00:00 0.0


2017-04-28 13:00:00 0.0


2017-04-28 14:00:00 0.0


2017-04-28 15:00:00 0.0


2017-04-28 16:00:00 0.0


2017-04-28 17:00:00 0.0


2017-04-28 18:00:00 0.0


2017-04-28 19:00:00 0.0


2017-04-28 20:00:00 0.0


2017-04-28 21:00:00 0.0


2017-04-28 22:00:00 0.0


2017-04-28 23:00:00 0.0


2017-04-29 00:00:00 0.0


2017-04-29 01:00:00 0.0


2017-04-29 02:00:00 0.0


2017-04-29 03:00:00 0.0


2017-04-29 04:00:00 0.0


2017-04-29 05:00:00 0.0


2017-04-29 0

2017-05-11 05:00:00 0.0


2017-05-11 06:00:00 0.0


2017-05-11 07:00:00 0.0


2017-05-11 08:00:00 0.0


2017-05-11 09:00:00 0.0


2017-05-11 10:00:00 0.0


2017-05-11 11:00:00 0.0


2017-05-11 12:00:00 0.0


2017-05-11 13:00:00 0.0


2017-05-11 14:00:00 0.0


2017-05-11 15:00:00 0.0


2017-05-11 16:00:00 0.0


2017-05-11 17:00:00 0.0


2017-05-11 18:00:00 0.0


2017-05-11 19:00:00 0.0


2017-05-11 20:00:00 0.0


2017-05-11 21:00:00 0.0


2017-05-11 22:00:00 0.0


2017-05-11 23:00:00 0.0


2017-05-12 00:00:00 0.0


2017-05-12 01:00:00 0.0


2017-05-12 02:00:00 0.0


2017-05-12 03:00:00 0.0


2017-05-12 04:00:00 0.0


2017-05-12 05:00:00 0.0


2017-05-12 06:00:00 0.0


2017-05-12 07:00:00 0.0


2017-05-12 08:00:00 0.0


2017-05-12 09:00:00 0.0


2017-05-12 10:00:00 0.0


2017-05-12 11:00:00 0.0


2017-05-12 12:00:00 0.0


2017-05-12 13:00:00 0.0


2017-05-12 14:00:00 0.0


2017-05-12 15:00:00 0.0


2017-05-12 16:00:00 0.0


2017-05-12 17:00:00 0.0


2017-05-12 18:00:00 0.0


2017-05-12 1

2017-05-24 10:00:00 0.0


2017-05-24 11:00:00 0.0


2017-05-24 12:00:00 0.0


2017-05-24 13:00:00 0.0


2017-05-24 14:00:00 0.0


2017-05-24 15:00:00 0.0


2017-05-24 16:00:00 0.0


2017-05-24 17:00:00 0.0


2017-05-24 18:00:00 0.0


2017-05-24 19:00:00 0.0


2017-05-24 20:00:00 0.0


2017-05-24 21:00:00 0.0


2017-05-24 22:00:00 0.0


2017-05-24 23:00:00 0.0


2017-05-25 00:00:00 0.0


2017-05-25 01:00:00 0.0


2017-05-25 02:00:00 0.0


2017-05-25 03:00:00 0.0


2017-05-25 04:00:00 0.0


2017-05-25 05:00:00 0.0


2017-05-25 06:00:00 0.0


2017-05-25 07:00:00 0.0


2017-05-25 08:00:00 0.0


2017-05-25 09:00:00 0.0


2017-05-25 10:00:00 0.0


2017-05-25 11:00:00 0.0


2017-05-25 12:00:00 0.0


2017-05-25 13:00:00 0.0


2017-05-25 14:00:00 0.0


2017-05-25 15:00:00 0.0


2017-05-25 16:00:00 0.0


2017-05-25 17:00:00 0.0


2017-05-25 18:00:00 0.6


2017-05-25 19:00:00 0.0


2017-05-25 20:00:00 0.675


2017-05-25 21:00:00 1.25


2017-05-25 22:00:00 0.05


2017-05-25 23:00:00 0.675


2017-0

...

2017-11-16 12:00:00 0.25


2017-11-16 13:00:00 0.45


2017-11-16 14:00:00 0.275


2017-11-16 15:00:00 0.375


2017-11-16 16:00:00 0.65


2017-11-16 17:00:00 1.0500001


2017-11-16 18:00:00 0.375


2017-11-16 19:00:00 0.0


2017-11-16 20:00:00 0.0


2017-11-16 21:00:00 0.0


2017-11-16 22:00:00 0.0


2017-11-16 23:00:00 0.0


2017-11-17 00:00:00 0.0


2017-11-17 01:00:00 0.0


2017-11-17 02:00:00 0.0


2017-11-17 03:00:00 0.0


2017-11-17 04:00:00 0.0


2017-11-17 05:00:00 0.0


2017-11-17 06:00:00 0.0


2017-11-17 07:00:00 0.0


2017-11-17 08:00:00 0.0


2017-11-17 09:00:00 0.0


2017-11-17 10:00:00 0.0


2017-11-17 11:00:00 0.0


2017-11-17 12:00:00 0.0


2017-11-17 13:00:00 0.0


2017-11-17 14:00:00 0.0


2017-11-17 15:00:00 0.0


2017-11-17 16:00:00 0.0


2017-11-17 17:00:00 0.0


2017-11-17 18:00:00 0.0


2017-11-17 19:00:00 0.0


2017-11-17 20:00:00 0.0


2017-11-17 21:00:00 0.0


2017-11-17 22:00:00 0.0


2017-11-17 23:00:00 0.0


2017-11-18 00:00:00 0.0


2017-11-18 01:00:00 0.0

2017-11-29 23:00:00 0.0


2017-11-30 00:00:00 0.0


2017-11-30 01:00:00 0.0


2017-11-30 02:00:00 0.0


2017-11-30 03:00:00 0.0


2017-11-30 04:00:00 0.0


2017-11-30 05:00:00 0.0


2017-11-30 06:00:00 0.0


2017-11-30 07:00:00 0.0


2017-11-30 08:00:00 0.0


2017-11-30 09:00:00 0.0


2017-11-30 10:00:00 0.0


2017-11-30 11:00:00 0.0


2017-11-30 12:00:00 0.0


2017-11-30 13:00:00 0.0


2017-11-30 14:00:00 0.0


2017-11-30 15:00:00 0.0


2017-11-30 16:00:00 0.0


2017-11-30 17:00:00 0.0


2017-11-30 18:00:00 0.0


2017-11-30 19:00:00 0.0


2017-11-30 20:00:00 0.0


2017-11-30 21:00:00 0.0


2017-11-30 22:00:00 0.0


2017-11-30 23:00:00 0.0


2017-12-01 00:00:00 0.0


2017-12-01 01:00:00 0.0


2017-12-01 02:00:00 0.0


2017-12-01 03:00:00 0.62500006


2017-12-01 04:00:00 2.2250001


2017-12-01 05:00:00 1.6000001


2017-12-01 06:00:00 0.0


2017-12-01 07:00:00 0.0


2017-12-01 08:00:00 0.0


2017-12-01 09:00:00 0.0


2017-12-01 10:00:00 0.0


2017-12-01 11:00:00 0.0


2017-12-01 12:00:00

2017-12-13 13:00:00 0.0


2017-12-13 14:00:00 0.0


2017-12-13 15:00:00 0.0


2017-12-13 16:00:00 0.0


2017-12-13 17:00:00 0.0


2017-12-13 18:00:00 0.0


2017-12-13 19:00:00 0.0


2017-12-13 20:00:00 0.0


2017-12-13 21:00:00 0.0


2017-12-13 22:00:00 0.0


2017-12-13 23:00:00 0.0


2017-12-14 00:00:00 0.0


2017-12-14 01:00:00 0.0


2017-12-14 02:00:00 0.0


2017-12-14 03:00:00 0.0


2017-12-14 04:00:00 0.0


2017-12-14 05:00:00 0.0


2017-12-14 06:00:00 0.0


2017-12-14 07:00:00 0.0


2017-12-14 08:00:00 0.0


2017-12-14 09:00:00 0.0


2017-12-14 10:00:00 0.0


2017-12-14 11:00:00 0.0


2017-12-14 12:00:00 0.0


2017-12-14 13:00:00 0.0


2017-12-14 14:00:00 0.0


2017-12-14 15:00:00 0.0


2017-12-14 16:00:00 0.0


2017-12-14 17:00:00 0.0


2017-12-14 18:00:00 0.0


2017-12-14 19:00:00 0.0


2017-12-14 20:00:00 0.0


2017-12-14 21:00:00 0.0


2017-12-14 22:00:00 0.0


2017-12-14 23:00:00 0.0


2017-12-15 00:00:00 0.0


2017-12-15 01:00:00 0.0


2017-12-15 02:00:00 0.0


2017-12-15 0

2017-12-27 08:00:00 0.0


2017-12-27 09:00:00 0.0


2017-12-27 10:00:00 0.0


2017-12-27 11:00:00 0.0


2017-12-27 12:00:00 0.0


2017-12-27 13:00:00 0.0


2017-12-27 14:00:00 0.0


2017-12-27 15:00:00 0.0


2017-12-27 16:00:00 0.0


2017-12-27 17:00:00 0.0


2017-12-27 18:00:00 0.0


2017-12-27 19:00:00 0.0


2017-12-27 20:00:00 0.0


2017-12-27 21:00:00 0.0


2017-12-27 22:00:00 0.0


2017-12-27 23:00:00 0.0


2017-12-28 00:00:00 0.0


2017-12-28 01:00:00 0.0


2017-12-28 02:00:00 0.0


2017-12-28 03:00:00 0.0


2017-12-28 04:00:00 0.0


2017-12-28 05:00:00 0.0


2017-12-28 06:00:00 0.0


2017-12-28 07:00:00 0.0


2017-12-28 08:00:00 0.0


2017-12-28 09:00:00 0.0


2017-12-28 10:00:00 0.0


2017-12-28 11:00:00 0.0


2017-12-28 12:00:00 0.0




PermissionError: [Errno 13] Permission denied: 'C:\\Users\\feder\\Documents\\CUAHSI\\repos\\Aquaholics_Anonymous/data/sleepers\\slee__2017_0401_1228_precipitation.csv'